In [ ]:
# ==========================================
# Smart Resource Allocation Simulator (Interactive + RL with Widgets)
# ==========================================

import numpy as np
import random
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import ipywidgets as widgets

# -----------------------
# 1. Environment
# -----------------------
class CityEnv:
    def __init__(self):
        self.reset()

    def reset(self):
        self.budget = 100
        self.state = np.array([50, 40, 30])  # traffic, health, education needs
        self.done = False
        return self.state

    def random_event(self):
        """Random events affecting budget or city state"""
        event_roll = np.random.rand()
        if event_roll < 0.1:  # 10% chance positive event
            bonus = random.randint(5, 20)
            self.budget += bonus
            print(f"🎉 Unexpected grant! Budget increased by {bonus} units.")
        elif event_roll > 0.9:  # 10% chance negative event
            loss = random.randint(5, 20)
            self.budget = max(0, self.budget - loss)
            print(f"⚠️ Disaster/maintenance cost! Budget decreased by {loss} units.")

        # Random state events
        state_event = np.random.rand()
        if state_event < 0.1:
            self.state[0] = min(100, self.state[0]+5)  # traffic spike
            print("🚗 Traffic spike due to festival! Traffic needs increased.")
        elif state_event > 0.9:
            self.state[1] = min(100, self.state[1]+5)  # health worsens
            print("🏥 Epidemic outbreak! Health needs increased.")

    def step(self, action):
        """
        action: 0 = invest in traffic
                1 = invest in hospitals
                2 = invest in schools
        """
        reward = 0
        if action == 0:
            self.state[0] = max(0, self.state[0]-5)
            reward = 10
        elif action == 1:
            self.state[1] = max(0, self.state[1]-5)
            reward = 8
        elif action == 2:
            self.state[2] = max(0, self.state[2]-5)
            reward = 7

        self.budget -= 10
        if self.budget <= 0:
            self.done = True

        # Apply random events
        self.random_event()
        return self.state, reward, self.done

# -----------------------
# 2. Q-Learning Agent
# -----------------------
class QAgent:
    def __init__(self, state_bins=21, action_size=3, lr=0.1, gamma=0.9, epsilon=0.2):
        self.state_bins = state_bins
        self.action_size = action_size
        self.q_table = np.zeros((state_bins, state_bins, state_bins, action_size))
        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon

    def discretize(self, state):
        return tuple((state//5).astype(int))

    def choose_action(self, state):
        s = self.discretize(state)
        if random.random() < self.epsilon:
            return random.randint(0, self.action_size-1)
        return np.argmax(self.q_table[s])

    def update(self, state, action, reward, next_state):
        s = self.discretize(state)
        ns = self.discretize(next_state)
        best_next = np.max(self.q_table[ns])
        self.q_table[s][action] += self.lr*(reward + self.gamma*best_next - self.q_table[s][action])

# -----------------------
# 3. Visualization
# -----------------------
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Create an output widget for the chart
plot_out = widgets.Output()
display(plot_out)

def plot_state(state, budget):
    with plot_out:
        clear_output(wait=True)  # only clears the plot, not widgets
        plt.figure(figsize=(6,4))
        bars = plt.bar(['Traffic', 'Health', 'Education'], state, color=['red','blue','green'])
        plt.ylim(0, 100)
        plt.title(f"City Status (Budget = {budget})")
        for bar,val in zip(bars,state):
            plt.text(bar.get_x()+0.25, val+1, str(val))
        plt.show()

# -----------------------
# 4. Train RL Agent
# -----------------------
env = CityEnv()
agent = QAgent()
episodes = 3000

for ep in range(episodes):
    state = env.reset()
    done = False
    while not done:
        action = agent.choose_action(state)
        next_state, reward, done = env.step(action)
        agent.update(state, action, reward, next_state)
        state = next_state

print("RL Agent training completed.")

# -----------------------
# 5. Interactive Widgets
# -----------------------
state = env.reset()
done = False

action_dropdown = widgets.Dropdown(
    options=[('Traffic',0), ('Hospitals',1), ('Schools',2), ('RL Agent Suggestion','agent'), ('Exit','exit')],
    description='Action:'
)

button = widgets.Button(description="Apply Action")

display(action_dropdown, button)

def on_button_clicked(b):
    global state, done
    if done:
        print("Simulation ended.")
        return

    user_input = action_dropdown.value

    if user_input == 'exit':
        done = True
        print("Simulation ended by user.")
        return

    if user_input == 'agent':
        action = agent.choose_action(state)
        print(f"🤖 RL Agent recommends: {['Traffic','Hospitals','Schools'][action]}")
    else:
        action = int(user_input)

    state, reward, done = env.step(action)
    print(f"Action applied. Reward earned: {reward}")
    plot_state(state, env.budget)

button.on_click(on_button_clicked)

Output()

Streaming output truncated to the last 5000 lines.
🚗 Traffic spike due to festival! Traffic needs increased.
⚠️ Disaster/maintenance cost! Budget decreased by 14 units.
🎉 Unexpected grant! Budget increased by 16 units.
🚗 Traffic spike due to festival! Traffic needs increased.
🎉 Unexpected grant! Budget increased by 15 units.
🎉 Unexpected grant! Budget increased by 16 units.
🚗 Traffic spike due to festival! Traffic needs increased.
🎉 Unexpected grant! Budget increased by 8 units.
⚠️ Disaster/maintenance cost! Budget decreased by 20 units.
🎉 Unexpected grant! Budget increased by 18 units.
🏥 Epidemic outbreak! Health needs increased.
🏥 Epidemic outbreak! Health needs increased.
⚠️ Disaster/maintenance cost! Budget decreased by 12 units.
⚠️ Disaster/maintenance cost! Budget decreased by 9 units.
🚗 Traffic spike due to festival! Traffic needs increased.
⚠️ Disaster/maintenance cost! Budget decreased by 20 units.
🏥 Epidemic outbreak! Health needs increased.
🏥 Epidemic outbreak! Health needs 

Dropdown(description='Action:', options=(('Traffic', 0), ('Hospitals', 1), ('Schools', 2), ('RL Agent Suggesti…

Button(description='Apply Action', style=ButtonStyle())

Action applied. Reward earned: 10
🤖 RL Agent recommends: Traffic
🚗 Traffic spike due to festival! Traffic needs increased.
Action applied. Reward earned: 10
Action applied. Reward earned: 8
Action applied. Reward earned: 8
🚗 Traffic spike due to festival! Traffic needs increased.
Action applied. Reward earned: 10
🚗 Traffic spike due to festival! Traffic needs increased.
Action applied. Reward earned: 7
